## Author- Daniel Melesse
- SML 310 project

### Import libraries

In [ ]:
import os
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import scipy
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation,RandomCrop, RandomContrast, Resizing,RandomZoom
import os
%matplotlib inline
np.random.seed(1)
import numpy as np
np.random.seed(5) 
import tensorflow as tf
#tf.set_random_seed(2)
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from time import perf_counter
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from IPython.display import Markdown, display

def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from time import perf_counter
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from IPython.display import Markdown, display
def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))

### Set up a working directory

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/
!pwd

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive


### Check GPU  and connect to it

In [ ]:
%tensorflow_version 2.x

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.8.0
/device:GPU:0
Num GPUs Available:  1


### Define train and test directories and load the data


- To make it easier, I created subfolders from 87000 images, and I put 2700 images for testing, and 21600 images for training and validaion. However, later I decided only to use 10368 images for training

In [ ]:
data_list = os.listdir('/content/drive/My Drive/Sign_Language/Dataset_2/ASL_test') 
train_dir = '/content/drive/My Drive/Sign_Language/Dataset_2/final/final_train/'
test_dir='/content/drive/My Drive/Sign_Language/Dataset_2/final/final_final_test/'

In [ ]:
directory_train= Path(train_dir)
image_paths = list(directory_train.glob(r'**/*.jpg'))
directory_test= Path(test_dir)
test_paths = list(directory_test.glob(r'**/*.jpg'))

### Explanatory Data Analysis

- Here, let's create a DataFrame with the path and the labels of the images of ASL dataset II so that we can use datagenerator in order to load the images in batch.

In [ ]:
def preprocess_image(path):
    labels = [str(path[i]).split("/")[-2] \
              for i in range(len(path))]
    image_path = pd.Series(path, name='image_path').astype(str)
    labels = pd.Series(labels, name='image_label')
    df = pd.concat([image_path, labels], axis=1)
    # This huffle the DataFrame and reset index
    image_data_frame = df.sample(frac=1,random_state=0).reset_index(drop = True)
    return image_data_frame

image_data_frame = preprocess_image(image_paths)
print('The number of train images in the ASL:', image_data_frame.shape[0])
print("")
print(' The total number of labels is:', len(image_data_frame.image_label.unique()))
print("")
print('The labels of the dataset are:', image_data_frame.image_label.unique())
print("")

The number of train images in the ASL: 21600

 The total number of labels is: 27

The labels of the dataset are: ['L' 'D' 'U' 'P' 'M' 'F' 'O' 'NOTHING' 'W' 'S' 'DELETE' 'B' 'G' 'C' 'T'
 'H' 'K' 'V' 'A' 'Q' 'I' 'R' 'Y' 'N' 'E' 'X' 'SPACE']



In [ ]:
image_data_frame.head(5)

,image_path,image_label
0,/content/drive/My Drive/Sign_Language/Dataset_...,L
1,/content/drive/My Drive/Sign_Language/Dataset_...,D
2,/content/drive/My Drive/Sign_Language/Dataset_...,U
3,/content/drive/My Drive/Sign_Language/Dataset_...,P
4,/content/drive/My Drive/Sign_Language/Dataset_...,M


In [ ]:
df_test = preprocess_image(test_paths)
print('The number of test images in the ASL dataset II is:', df_test.shape[0])
print("")
print(' The total number of labels is:', len(df_test.image_label.unique()))

The number of test images in the ASL dataset II is: 2700

 The total number of labels is: 27


- As we can see above, there are 21000 images, however we want 1037 images for training, and there are 27 classes as we want the best pretrained model.


#### Display sample images from the dataset with thier label

- Here let's plot a few images randomly from the training set

- Given, I have 87000 images and the images are colored and have high resolution, I am going to use only a subset of it which is 10368 for training, 1036 for validation, and 2700 for testing. 

- Now, let's use subset of the training data in order to train pretrain models to select the best pretrained models. Let's use just use 10 of 10368 images which is 1037 images.

In [ ]:
# Split into training and test datasets
train_df, test_df = train_test_split(image_data_frame.sample(frac = 0.06004), test_size=0.2, random_state=0)
print(train_df.shape)

(1037, 2)


#### Progressive loading

- Here we define our data generators for training, validation, and test

- This saves huge computational cost, I am loading it from the dataframe to save computation time rather than loading it from a directory. It loads images in batch.

- Depending on the type of transfer learning I fix the size of the image.

- Here we will normalize each image by dividing by 255. In addition, we further split the training set to train and validation. Where the validation set will be 10% of the original training set. The color mode is RGB as we have colored images, plus the batch_size is 32, and the class mode is catagorial.

In [ ]:
num_class=len(data_list)
batch_size=32
train_datagen=ImageDataGenerator(
    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
    height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
    horizontal_flip=False, vertical_flip=False, rescale=1.0/255.0,
    preprocessing_function=None, data_format=None, validation_split=0.1, dtype=None
)

test_datagen=ImageDataGenerator(
    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
    height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
    horizontal_flip=False, vertical_flip=False, rescale=1/255,
    preprocessing_function=None, data_format=None, validation_split=0, dtype=None
)
train_data = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='image_path',
        y_col='image_label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True,
        seed=0,
        subset='training'
    )
val_data = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='image_path',
        y_col='image_label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True,
        seed=0,
        subset='validation')
test_data = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col='image_path',
        y_col='image_label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=False
    )

Found 934 validated image filenames belonging to 27 classes.
Found 103 validated image filenames belonging to 27 classes.
Found 260 validated image filenames belonging to 27 classes.


#### Neural Network

In [ ]:
#adapted from https://www.tensorflow.org/tutorials/images/data_augmentation
def random_invert_img(x, p=0.5):
    if  tf.random.uniform([]) < p:
        x = (255-x)
    else:
        x
    return x


def random_invert(factor=0.5):
    return layers.Lambda(lambda x: random_invert_img(x, factor))

random_invert = random_invert()


class RandomInvert(layers.Layer):
    def __init__(self, factor=0.5, **kwargs):
        super().__init__(**kwargs)
        self.factor = factor

    def call(self, x):
        return random_invert_img(x)

In [ ]:
def random_change_bright_img(x):
    return tf.image.random_brightness(x, 0.2)

def random_change_brightness():
    return layers.Lambda(lambda x: random_change_bright_img(x))

random_change_brightness = random_change_brightness()


class RandomBrightness(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, x):
        return random_change_brightness (x)

In [ ]:
def data_augmenter():
    '''
    Sequential model composed of 2 layers
    '''
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip("horizontal"))
    data_augmentation.add(RandomRotation(0.1)) 
    return data_augmentation
def data_augmenter2():
    '''
    Sequential model composed of 3 layers
    '''
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip("horizontal_and_vertical"))
    data_augmentation.add(RandomBrightness())
    data_augmentation.add(RandomRotation(0.2)) 

    return data_augmentation

def data_augmenter3():
    '''
    Sequential model composed of 3 layers
    '''
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomRotation(0.3)) 
    data_augmentation.add(RandomBrightness())
    data_augmentation.add(RandomContrast(0.2))
    return data_augmentation
def data_augmenter4():
    '''
    Sequential model composed of 4 layers
    '''
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip("horizontal"))
    data_augmentation.add(RandomRotation(0.4)) 
    data_augmentation.add(RandomBrightness())
    data_augmentation.add(RandomContrast(0.2))
    data_augmentation.add(RandomInvert())

    return data_augmentation

def data_augmenter5():
    '''
    Sequential model composed of 2 layers
    '''
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip("horizontal"))
    data_augmentation.add(RandomRotation(0.2)) 
    data_augmentation.add(RandomBrightness())
    data_augmentation.add(RandomCrop(20,20))
    data_augmentation.add(Resizing(28, 28, interpolation='bilinear', crop_to_aspect_ratio=True)) #resize to 28 x 28

    return data_augmentation

In [ ]:
def best_pretrained_model(model):
    kwargs =    {'input_shape':(224, 224, 3),
                'include_top':False,
                'weights':'imagenet',
                'pooling':'avg'}
    pretrained_model = model(**kwargs)
    pretrained_model.trainable = False
    inputs = pretrained_model.input
   # x=data_augmenter3()(inputs) if data augumentation, have to rearrage how we set the input and output from the pretrained model
    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x= Dropout(0.2)(x)
    x= BatchNormalization()(x)
    x= tf.keras.layers.Dense(64, activation='relu')(x)
    x= Dropout(0.2)(x)
    x= BatchNormalization()(x)
    outputs = tf.keras.layers.Dense(27, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
all_models = {
    "MobileNet": {"model":tf.keras.applications.MobileNet, "perf":0},
    "VGG19": {"model":tf.keras.applications.VGG19, "perf":0},
    "EfficientNetB7": {"model":tf.keras.applications.EfficientNetB7, "perf":0},
    "InceptionV3": {"model":tf.keras.applications.InceptionV3, "perf":0},
    "Xception": {"model":tf.keras.applications.Xception, "perf":0},
    "DenseNet121": {"model":tf.keras.applications.DenseNet121, "perf":0},
    "ResNet152": {"model":tf.keras.applications.ResNet152, "perf":0}}
for name, model in all_models.items():
    best_models = best_pretrained_model(model['model'])
    all_models[name]['model'] = best_models
    history = best_models.fit(train_data,validation_data=val_data,epochs=5,verbose=0) 
    val_acc = history.history['val_accuracy']
    all_models[name]['val_acc'] = [round(v,4) for v in val_acc]

In [ ]:
final = []
for name, v in all_models.items():
    final.append([ name, all_models[name]['val_acc'][-1]])
dataf = pd.DataFrame(final, 
                          columns = ['model','val_accuracy'])
dataf.sort_values(by='val_accuracy', ascending=False, inplace=True)
dataf.reset_index(inplace=True,drop=True)

In [ ]:
plt.figure(figsize = (8,5))
sns.barplot(x = 'model', y = 'val_accuracy', data = dataf)
plt.title('Accuracy on the validation set for 5 epoch)', fontsize = 15)
plt.ylim(0,1)
plt.xticks(rotation=90)
plt.show()